In [ ]:
import holodeck as holo
from holodeck import single_sources, utils, plot, detstats
from holodeck.constants import YR

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

import hasasia.sensitivity as hsen
import hasasia.sim as hsim
import hasasia.skymap as hsky

# Read in saved sam & strain data

Option 1) Make realizations

In [ ]:
# dur = 15 * YR
# nfreqs = 40
# cad = dur/nfreqs/2
# fobs= utils.nyquist_freqs(dur, cad)
# fobs_edges = utils.nyquist_freqs_edges(dur, cad)
# df_orb = np.diff(fobs_edges/2)
# print(len(fobs))
# sam = holo.sam.Semi_Analytic_Model()
# hc_ss, hc_bg = sam.ss_gwb(fobs_edges, realize=3, loudest=10000)
# np.savez('/Users/emigardiner/GWs/holodeck/ecg-notebooks/detstats_functions/npz_arrays/sam02_r3_f40_l10000.npz', 
#          dur=dur, cad=cad, fobs=fobs, df_orb=df_orb, hc_ss=hc_ss, hc_bg=hc_bg, edges=sam.edges)

Option 2) Load existing

In [ ]:
# sam02 has r3, f40, 15 yr dur, 40 freqs, ca
hcfile = np.load('/Users/emigardiner/GWs/holodeck/ecg-notebooks/detstats_functions/npz_arrays/sam02_r3_f40_l10000.npz')
dur = hcfile['dur']
cad = hcfile['cad']
fobs = hcfile['fobs'][:]
df_orb = hcfile['df_orb']
hc_ss = hcfile['hc_ss']
hc_bg = hcfile['hc_bg']
hcfile.close()

shape = hc_ss.shape
F, R, L = shape[0], shape[1], shape[2]
print('F, R, L =', F,R,L)
sam_name = 'sam02: dur=%.2f yr, cad=%.4f yr, %d loudest' % (dur/YR, cad/YR, L)
print(sam_name)

# Build skymap

In [ ]:
# Random sky positions
NSIDE1 = 32
print( "Approximate resolution at NSIDE {} is {:.2} deg".format(
        NSIDE1, hp.nside2resol(NSIDE1, arcmin=True) / 60))
NPIX1 = hp.nside2npix(NSIDE1)
print(NPIX1)

# Random sky positions
NSIDE2 = 28
print( "Approximate resolution at NSIDE {} is {:.2} deg".format(
        NSIDE2, hp.nside2resol(NSIDE2, arcmin=True) / 60))
NPIX2 = hp.nside2npix(NSIDE2)
print(NPIX2)

# Random sky positions
NSIDE3 = 24
print( "Approximate resolution at NSIDE {} is {:.2} deg".format(
        NSIDE2, hp.nside2resol(NSIDE3, arcmin=True) / 60))
NPIX3 = hp.nside2npix(NSIDE3)
print(NPIX3)

# Place Random Single Sources

In [ ]:
m_strain1 = np.zeros((F,NPIX1)) # frequency, pixel positon
m_strain2 = np.zeros((F,NPIX2)) # frequency, pixel positon
m_strain3 = np.zeros((F,NPIX2)) # frequency, pixel positon
# paint the background evenly across
rr=0 # just one realization
# avg_pix_strain = hc_bg[:] #/ NPIX # (F,R)

# strain per pixel = sqrt(hc^2 / npixels)
m_strain1 = np.ones((F,NPIX1)) * hc_bg[:,rr,np.newaxis]/np.sqrt(NPIX1)
m_strain2 = np.ones((F,NPIX2)) * hc_bg[:,rr,np.newaxis]/np.sqrt(NPIX2)
m_strain3 = np.ones((F,NPIX3)) * hc_bg[:,rr,np.newaxis]/np.sqrt(NPIX3)
# print('hc_bg=\n',hc_bg[:,rr], '\nnpix=',NPIX1, '\nbgpix strain=\n', (m_strain1[:,0]))
# print('hc_bg=\n',hc_bg[:,rr], '\nnpix=',NPIX2, '\nbgpix strain=\n', (m_strain2[:,0]))

# choose random pixels to place single sources, 
pix_ss1 = np.random.randint(0, NPIX1-1, size=F*L).reshape(F,L)
pix_ss2 = np.random.randint(0, NPIX2-1, size=F*L).reshape(F,L)
pix_ss3 = np.random.randint(0, NPIX3-1, size=F*L).reshape(F,L)
print(pix_ss1.shape)

# and add the ss_strains to their pixels
for ff in range(F):
    for ll in range(L):
        m_strain1[ff,pix_ss1[ff,ll]] = np.sqrt(m_strain1[ff,pix_ss1[ff,ll]]**2 + hc_ss[ff,rr,ll]**2)
        m_strain2[ff,pix_ss2[ff,ll]] = np.sqrt(m_strain2[ff,pix_ss2[ff,ll]]**2 + hc_ss[ff,rr,ll]**2)
        m_strain3[ff,pix_ss3[ff,ll]] = np.sqrt(m_strain3[ff,pix_ss3[ff,ll]]**2 + hc_ss[ff,rr,ll]**2)

In [ ]:
# show in skymap
# ff=23
def ff_mollview(hc_ss, pix_ss, m_strain1, ff, plot=False, saveloc=None, ax=None):
    # print(hc_ss[ff,rr,:])
    if plot:
        plt.scatter(np.arange(len(hc_ss[ff,rr,:])), pix_ss[ff,:], c=np.log(hc_ss[ff,rr,:]), cmap='viridis', s=0.5)
        plt.colorbar(label='log(hc_ss)')
        plt.xlabel("L'th Loudest")
        plt.ylabel('Pixel')
    if ax is not None:
        plt.axes(ax)
    hp.mollview(m_strain1[ff], title='%d Loudest Single Sources at %.2f yr$^{-1}$ (freq bin %d/%d)' % (L, fobs[ff]*YR, ff, F))
    if saveloc is not None:
        fname = saveloc+'hp_ff%02d.png' % ff
        print(fname)
        plt.savefig(fname, dpi=300)
    plt.close()
# fig = ff_mollview(hc_ss, pix_ss, m_strain1, 23, plot=True)


## Per Frequency Skymaps

In [ ]:
# saveloc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/healpix/2023-05-13_sam02_l10000/'
for ff in range(F):
    fig = ff_mollview(hc_ss, pix_ss2, m_strain1, ff, saveloc=None)
    # fig.savefig(saveloc+'hp_ff%02d.png' % ff, dpi=300)
    # plt.close(fig)

In [ ]:
fig, axs = plt.subplots(nrows=10, ncols=int(F/10))
for ff in range(F):
    title='$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
    row, col = int(ff/4), ff%4
    # print(row,col)
    plt.axes(axs[row,col])
    hp.mollview(m_strain2[ff], title=title, hold=True)

fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
fig.set_size_inches(18, 30)
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

# Spherical Harmonics

freq bin 5 is nice, lets use that first

In [ ]:
Cl_lmax1 = np.zeros((F,9))
Cl_lmax2 = np.zeros((F,9))
Cl_lmax3 = np.zeros((F,9))
# E_l_lmax8 = np.zeros((F,9))

In [ ]:
for ff in range(F):
    # power spectrum of a healpix map
    Cl_lmax1[ff,:] = hp.anafast(m_strain1[ff], lmax=8)
    Cl_lmax2[ff,:] = hp.anafast(m_strain2[ff], lmax=8)
    Cl_lmax3[ff,:] = hp.anafast(m_strain3[ff], lmax=8)
print(Cl_lmax1.shape)

Cl_C0_ratio1 = Cl_lmax1[:,:]/Cl_lmax1[:,0][:,np.newaxis]
Cl_C0_ratio2 = Cl_lmax2[:,:]/Cl_lmax2[:,0][:,np.newaxis]
Cl_C0_ratio3 = Cl_lmax3[:,:]/Cl_lmax3[:,0][:,np.newaxis]
print(Cl_C0_ratio1.shape)

l_vals = np.arange(len(Cl_lmax1[0]))

Cl_cum1 = np.sum(Cl_lmax1[:,1:], axis=1)
Cl_cum_ratio1 = Cl_cum1/Cl_lmax1[:,0]
Cl_cum2 = np.sum(Cl_lmax2[:,1:], axis=1)
Cl_cum_ratio2 = Cl_cum2/Cl_lmax2[:,0]
Cl_cum3 = np.sum(Cl_lmax3[:,1:], axis=1)
Cl_cum_ratio3 = Cl_cum3/Cl_lmax3[:,0]
print(Cl_cum_ratio1.shape)

plot all the spherical harmonics

In [ ]:
fig, axs = plt.subplots(nrows=10, ncols=int(F/10), sharex=True, sharey=True)
for ff in range(F):
    title='$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
    row, col = int(ff/4), ff%4
    # print(row,col)
    ax = axs[row,col]
    ax.plot(l_vals, Cl_C0_ratio1[ff])
    if row==9:
        ax.set_xlabel('$l$')
    if col==0:
        ax.set_ylabel('$C_l/C_0$')

# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
# fig.tight_layout()
fig.set_size_inches(18, 30)
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

In [ ]:
import matplotlib.cm as cm
colors=cm.rainbow(np.linspace(0,1,F))
fobs_nHz = fobs*10**9
print(fobs_nHz)

Plot Just Cl

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(l_vals[:], Cl_lmax1[ff,:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
ax.set_xlabel('$l$')
ax.set_ylabel('$C_l$')
ax.set_yscale('log')
ax.set_title('Spherical Harmonic Coefficients, NPIX=%d' % NPIX1)
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(l_vals[:], Cl_lmax2[ff,:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
ax.set_xlabel('$l$')
ax.set_ylabel('$C_l$')
ax.set_yscale('log')
ax.set_title('Spherical Harmonic Coefficients, NPIX = %d' % NPIX2)
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(l_vals[:], Cl_lmax3[ff,:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
ax.set_xlabel('$l$')
ax.set_ylabel('$C_l$')
ax.set_yscale('log')
ax.set_title('Spherical Harmonic Coefficients, NPIX = %d' % NPIX3)
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(12,5))

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax1.plot(l_vals[1:], Cl_lmax1[ff,1:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
    ax2.plot(l_vals[1:], Cl_lmax2[ff,1:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
    ax3.plot(l_vals[1:], Cl_lmax3[ff,1:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
for ax in (ax1, ax2, ax3):
    ax.set_xlabel('$\ell$')
    ax.set_yscale('log')
ax1.set_ylabel('$C_l$')
ax1.set_title('NPIX = %d' %NPIX1)
ax2.set_title('NPIX = %d' %NPIX2)
ax3.set_title('NPIX = %d' %NPIX3)
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

### plot C_l/C_0

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(l_vals[1:], Cl_C0_ratio1[ff,1:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
ax.set_xlabel('$\ell$')
ax.set_ylabel('$C_\ell/C_0$')
ax.set_yscale('log')
ax.set_title('Spherical Harmonic Coefficients, NPIX = %d' %NPIX1)
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(l_vals[1:], Cl_C0_ratio2[ff,1:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
ax.set_xlabel('$\ell$')
ax.set_ylabel('$C_\ell/C_0$')
ax.set_yscale('log')
ax.set_title('Spherical Harmonic Coefficients, NPIX = %d' %NPIX2)
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(l_vals[1:], Cl_C0_ratio3[ff,1:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
ax.set_xlabel('$\ell$')
ax.set_ylabel('$C_\ell/C_0$')
ax.set_yscale('log')
ax.set_title('Spherical Harmonic Coefficients, NPIX = %d' %NPIX3)
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(12,5))

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax1.plot(l_vals[1:], Cl_C0_ratio1[ff,1:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
    ax2.plot(l_vals[1:], Cl_C0_ratio2[ff,1:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
    ax3.plot(l_vals[1:], Cl_C0_ratio3[ff,1:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
for ax in (ax1, ax2, ax3):
    ax.set_xlabel('$\ell$')
    ax.set_yscale('log')
ax1.set_ylabel('$C_\ell/C_0$')
ax1.set_title('NPIX = %d' %NPIX1)
ax2.set_title('NPIX = %d' %NPIX2)
ax3.set_title('NPIX = %d' %NPIX3)
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

### Compare to energy
fraction of energy in the background vs the sum over the 1000 sources.

"I’m curious if the ratio of power in the harmonics (i.e. top-right plot) is the same as the fraction of energy in the background vs the sum over the 1000 sources."

$S_h(f) \propto \Omega_{gw}(f) / f^3\quad (55)$    so    $\Omega_{gw}(f) \propto S_h(f) \times f^3$

$h_c(f) \propto \sqrt{f S_h(f)} \quad (56)$    so    $S_h(f) \propto h_c^2(f) /f$

$$ \Omega_{gw}(f) \propto S_h(f) \times f^3 \propto (h_c^2(f) /f) \times f^3 \propto h_c^2(f) \times f^2$$

There's also a factor of $H_0^2$ which has units of time, and would cancel with the $f$ units to get dimensionless energy density.

none of this matters because if we're looking for the ratio of energy at each frequency, the frequencies cancel

In [ ]:
print(m_strain1[0].size)
print(NSIDE1, NPIX1)

In [ ]:
hc_ratio = np.sqrt(np.sum(hc_ss**2, axis=-1))/hc_bg
print(hc_ratio.shape)

fig, (ax, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(8,6), gridspec_kw={'width_ratios': [4, 1]})

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(l_vals[1:], Cl_C0_ratio1[ff,1:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
    ax2.axhline(hc_ratio[ff,0], color=colors[ff], alpha=0.5)
ax.set_xlabel('$\ell$')
ax.set_ylabel('$C_\ell/C_0$')
ax.set_yscale('log')
ax2.set_yscale('log')
ax.set_title('Spherical Harmonic Coefficients')
ax.legend()

ax2.set_title('$h_{c,10,000\ \mathrm{loudest}}/h_{c,\mathrm{bg}}$')
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)

fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

### Plot l(l+1)C_l

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
xx = l_vals[1:]
yy = (l_vals*(l_vals+1))[np.newaxis,1:]*Cl_lmax1[:,1:]

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(xx, yy[ff], c=colors[ff], alpha=0.5, label=label,
            marker='o')
ax.set_xlabel('$\ell$')
ax.set_ylabel('$\ell(\ell+1)C_\ell$')
ax.set_yscale('log')
ax.set_title('(Not Normalized) Spherical Harmonics')
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

## Plot C_l>0/C_0 vs. f

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
xx = fobs*YR
yy = Cl_cum_ratio1

ax.plot(xx, yy, c='k', alpha=0.5, marker='o', label=sam_name)
ax.set_xlabel('Observed GW Frequency, $f$ (1/yr)')
ax.set_ylabel('$C_{\ell>0}/C_0$', fontsize=12)
ax.set_yscale('log')
# ax.set_title('(Not Normalized) Spherical Harmonics')
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

## normalizing

In [ ]:
# normalizing each frequency strains by total strain of each frequency
hc2_tot = np.sum(m_strain1**2, axis=1)
print(hc2_tot.shape)
print(m_strain1.shape)
m_strain1_norm = m_strain1/np.sqrt(hc2_tot[:,np.newaxis])

Cl_norm=np.zeros((F,9))
for ff in range(F):
    # power spectrum of a healpix map
    Cl_norm[ff,:] = hp.anafast(m_strain1_norm[ff], lmax=8)

Cl_C0_ratio_norm = Cl_norm/Cl_norm[:,0,np.newaxis]

#### Plot l(l+1)C_l

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
xx = l_vals[1:]
yy = (l_vals*(l_vals+1))[np.newaxis,1:]*Cl_norm[:,1:]

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(xx, yy[ff], c=colors[ff], alpha=0.5, label=label,
            marker='o')
ax.set_xlabel('$\ell$')
ax.set_ylabel('$\ell(\ell+1)C_\ell$')
ax.set_yscale('log')
ax.set_title('Normalized Spherical Harmonics')
ax.legend(loc='lower right')
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

# Rebuild maps from harmonics

In [ ]:
print(Cl_lmax1.shape)

In [ ]:
m_sph1 = np.zeros_like(m_strain1)
for ff in range(F):
    m_sph1[ff,:] = hp.synfast(Cl_lmax1[ff], NSIDE1, lmax=8)

In [ ]:
fig, axs = plt.subplots(nrows=10, ncols=int(F/10))
for ff in range(F):
    title='$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
    row, col = int(ff/4), ff%4
    # print(row,col)
    plt.axes(axs[row,col])
    hp.mollview(m_sph1[ff], title=title, hold=True)

fig.suptitle('%d Loudest Single Sources, Recovered from Spherical Harmonics' % L, y=.895)
fig.tight_layout()
fig.set_size_inches(18, 30)
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig